In [4]:
# general
import datetime as dt
# data 
import xarray as xr 
import numpy as np
import pandas as pd
# plotting
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio
# helper tools
from utils.helper_funcs import create_windrose_df
import utils.helper_funcs as hf
from metpy import calc, units
# make plotly work 
init_notebook_mode(connected=True)
cf.go_offline()

ModuleNotFoundError: No module named 'utils'

In [2]:
# Import SAIL met data
winter_22 = slice('2021-12-01', '2022-03-31')
winter_23 = slice('2022-12-01', '2023-03-31')
# Set the location of the data folder where this data will be stored
winter_22_folder = 'winter_21_22'
winter_23_folder = 'winter_22_23'
# change to location of data folder on your machine
storage_directory = f'/storage/dlhogan/synoptic_sublimation/sail_data'

w22_sail_met_ds = xr.open_dataset(f'{storage_directory}/{winter_22_folder}/met_20211001_20220930.nc')
w23_sail_met_ds = xr.open_dataset(f'{storage_directory}/{winter_23_folder}/met_20221001_20230930.nc')

# load the kettle ponds data
w22_sail_kettle_ponds = xr.open_dataset(f'{storage_directory}/{winter_22_folder}/eddy_covariance_kettle_ponds_noqc_20211001_20220930.nc')
w23_sail_kettle_ponds = xr.open_dataset(f'{storage_directory}/{winter_23_folder}/eddy_covariance_kettle_ponds_noqc_20221001_20230930.nc')

# filter to only include the winter months
w22_sail_met_ds = w22_sail_met_ds.sel(time=winter_22)
w23_sail_met_ds = w23_sail_met_ds.sel(time=winter_23)
w22_sail_kettle_ponds = w22_sail_kettle_ponds.sel(time=winter_22)
w23_sail_kettle_ponds = w23_sail_kettle_ponds.sel(time=winter_23)

VARIABLES = ['atmos_pressure','temp_mean', 'rh_mean','vapor_pressure_mean','wspd_arith_mean','wdir_vec_mean']
VARIABLES_KP = ['mean_u','mean_v','mean_t','mean_q','atm_pres','wind_spd','wind_dir','lv']
# qc the data
w22_sail_met_qc_ds = hf.qc_sail_met(w22_sail_met_ds,VARIABLES, 'temp_mean', 'rh_mean', 'wspd_arith_mean', 'vapor_pressure_mean')
w23_sail_met_qc_ds = hf.qc_sail_met(w23_sail_met_ds,VARIABLES, 'temp_mean', 'rh_mean', 'wspd_arith_mean', 'vapor_pressure_mean')
w23_sail_met_kp_qc_ds = hf.qc_sail_met(w23_sail_kettle_ponds, VARIABLES_KP, 'mean_t', 'mean_q', 'wind_spd', 'mean_q')
w22_sail_met_kp_qc_ds = hf.qc_sail_met(w22_sail_kettle_ponds, VARIABLES_KP, 'mean_t', 'mean_q', 'wind_spd', 'mean_q')

atmos_pressure has 0 nan values
temp_mean has 0 nan values
rh_mean has 0 nan values
vapor_pressure_mean has 0 nan values
wspd_arith_mean has 8 nan values
wdir_vec_mean has 0 nan values
-------------------
atmos_pressure has 0 nan values
temp_mean has 0 nan values
rh_mean has 0 nan values
vapor_pressure_mean has 0 nan values
wspd_arith_mean has 0 nan values
wdir_vec_mean has 0 nan values
-------------------
mean_u has 0 nan values
mean_v has 0 nan values
mean_t has 5459 nan values
mean_q has 5459 nan values
atm_pres has 5459 nan values
wind_spd has 0 nan values
wind_dir has 0 nan values
lv has 0 nan values
-------------------
mean_u has 2079 nan values
mean_v has 2079 nan values
mean_t has 5788 nan values
mean_q has 5788 nan values
atm_pres has 5788 nan values
wind_spd has 2079 nan values
wind_dir has 2079 nan values
lv has 2079 nan values
-------------------


In [3]:
# calculat estimate for vapor pressure deficit
w22_sail_met_qc_ds['sat_vapor_pressure'] = calc.saturation_vapor_pressure(w22_sail_met_qc_ds['temp_mean'])
w23_sail_met_qc_ds['sat_vapor_pressure'] = calc.saturation_vapor_pressure(w23_sail_met_qc_ds['temp_mean'])

w22_sail_met_qc_ds['vapor_pressure_deficit'] = w22_sail_met_qc_ds['sat_vapor_pressure'] - (w22_sail_met_qc_ds['vapor_pressure_mean']*units.units.kilopascal)
w23_sail_met_qc_ds['vapor_pressure_deficit'] = w23_sail_met_qc_ds['sat_vapor_pressure'] - w23_sail_met_qc_ds['vapor_pressure_mean']*units.units.kilopascal


In [4]:
# get preciptiation data
w22_all_precip = pd.read_csv('../01_data/processed_data/precipitation/w22_all_precip_30min.csv', index_col=0, parse_dates=True)
w23_all_precip = pd.read_csv('../01_data/processed_data/precipitation/w23_all_precip_30min.csv', index_col=0, parse_dates=True)

# get pluvio data
w22_pluvio = w22_all_precip['SAIL_gts_pluvio'].loc[winter_22]
w23_pluvio = w23_all_precip['SAIL_gts_pluvio'].loc[winter_23]

In [5]:
# get sublimation data
w22_sublimation = pd.read_csv('../01_data/processed_data/sublimation/w22_winter_sublimation.csv', index_col=0, parse_dates=True)
w23_sublimation = pd.read_csv('../01_data/processed_data/sublimation/w23_winter_sublimation.csv', index_col=0, parse_dates=True)

# filter to winter
w22_sublimation = w22_sublimation.loc[winter_22]
w23_sublimation = w23_sublimation.loc[winter_23]

In [6]:
# make the index the day of water year
dowy = lambda x: x.dayofyear - 274 if x.dayofyear >= 274 else x.dayofyear + 91

def create_comparable_df_from_ds(ds, variable, agg_func='mean'):
    if agg_func == 'mean':
        df_var = ds[variable].groupby('time.date').mean().to_dataframe().reset_index()
    elif agg_func == 'max':
        df_var = ds[variable].groupby('time.date').max().to_dataframe().reset_index()
    else:
        raise ValueError('agg_func must be mean or max')
    df_var.index = pd.to_datetime(df_var['date']).apply(dowy)
    df_var['date'] = pd.to_datetime(df_var['date'])
    df_var.index.name = 'dowy'
    df_var = df_var.loc[0:]
    return df_var

def create_comparable_df_from_df(df):
    df_var = df.groupby(df.index.date).sum().reset_index()
    df_var.index = pd.to_datetime(df_var['index']).apply(dowy)
    df_var['date'] = pd.to_datetime(df_var['index'])
    df_var.index.name = 'dowy'
    df_var = df_var.loc[0:]
    return df_var

# create comparable dataframes for each variable
w22_mean_wspd = create_comparable_df_from_ds(w22_sail_met_qc_ds, 'wspd_arith_mean', 'mean')
w23_mean_wspd = create_comparable_df_from_ds(w23_sail_met_qc_ds, 'wspd_arith_mean', 'mean')
w22_max_wspd = create_comparable_df_from_ds(w22_sail_met_qc_ds, 'wspd_arith_mean', 'max')
w23_max_wspd = create_comparable_df_from_ds(w23_sail_met_qc_ds, 'wspd_arith_mean', 'max')
w22_mean_temperture = create_comparable_df_from_ds(w22_sail_met_qc_ds, 'temp_mean', 'mean')
w23_mean_temperture = create_comparable_df_from_ds(w23_sail_met_qc_ds, 'temp_mean', 'mean')
w22_mean_vapor_pressure_deficit = create_comparable_df_from_ds(w22_sail_met_qc_ds, 'vapor_pressure_deficit', 'mean')
w23_mean_vapor_pressure_deficit = create_comparable_df_from_ds(w23_sail_met_qc_ds, 'vapor_pressure_deficit', 'mean')

# Precipitation
w22_cumulative_precip = create_comparable_df_from_df(w22_pluvio)['SAIL_gts_pluvio'].cumsum()
w23_cumulative_precip = create_comparable_df_from_df(w23_pluvio)['SAIL_gts_pluvio'].cumsum()

# Sublimation
w22_cumulative_sublimation = create_comparable_df_from_df(w22_sublimation)['sublimation_prcp_0_5_mm'].cumsum()
w23_cumulative_sublimation = create_comparable_df_from_df(w23_sublimation)['sublimation_prcp_0_5_mm'].cumsum()

In [7]:
# print the mean for each variable
print(f'Winter 21/22 Mean Wind Speed: {w22_mean_wspd["wspd_arith_mean"].mean()}')
print(f'Winter 22/23 Mean Wind Speed: {w23_mean_wspd["wspd_arith_mean"].mean()}')
print(f'Winter 21/22 Max Wind Speed: {w22_max_wspd["wspd_arith_mean"].max()}')
print(f'Winter 22/23 Max Wind Speed: {w23_max_wspd["wspd_arith_mean"].max()}')
print(f'Winter 21/22 Mean Temperature: {w22_mean_temperture["temp_mean"].mean()}')
print(f'Winter 22/23 Mean Temperature: {w23_mean_temperture["temp_mean"].mean()}')
print(f'Winter 21/22 Mean Vapor Pressure: {w22_mean_vapor_pressure_deficit["vapor_pressure_deficit"].mean()}')
print(f'Winter 22/23 Mean Vapor Pressure: {w23_mean_vapor_pressure_deficit["vapor_pressure_deficit"].mean()}')


Winter 21/22 Mean Wind Speed: 2.2462756633758545
Winter 22/23 Mean Wind Speed: 2.2068264484405518
Winter 21/22 Max Wind Speed: 36.63999938964844
Winter 22/23 Max Wind Speed: 19.360000610351562
Winter 21/22 Mean Temperature: -6.817960739135742
Winter 22/23 Mean Temperature: -8.203235626220703
Winter 21/22 Mean Vapor Pressure: 164.3073272705078
Winter 22/23 Mean Vapor Pressure: 127.32646179199219


In [11]:
# 2022 wind rose data
wspd_var = 'wind_spd'
wdir_var = 'wind_dir'
# create a windrose dataframe
w22_wind_rose_df = create_windrose_df(w22_sail_met_kp_qc_ds.to_dataframe()[[wspd_var, wdir_var]], wind_dir_var=wdir_var, wind_spd_var=wspd_var)
w23_wind_rose_df = create_windrose_df(w23_sail_met_kp_qc_ds.to_dataframe()[[wspd_var, wdir_var]], wind_dir_var=wdir_var, wind_spd_var=wspd_var)

w22_wind_rose_dir_df = w22_wind_rose_df[['direction','frequency']].groupby(['direction'], observed=True).sum().reset_index()
w23_wind_rose_dir_df = w23_wind_rose_df[['direction','frequency']].groupby(['direction'], observed=True).sum().reset_index()

# wind rose data for winds greater than 5 m/s
w22_wind_rose_df['speed_vals'] = (w22_wind_rose_df['speed'].cat.codes +1)*2
w23_wind_rose_df['speed_vals'] = (w23_wind_rose_df['speed'].cat.codes +1)*2

w22_wind_rose_spd_df = w22_wind_rose_df[w22_wind_rose_df['speed_vals']>=5][['direction','frequency']].groupby(['direction'], observed=True).sum().reset_index()
w23_wind_rose_spd_df = w23_wind_rose_df[w23_wind_rose_df['speed_vals']>=5][['direction','frequency']].groupby(['direction'], observed=True).sum().reset_index()

/home/dlhogan/GitHub/Synoptic-Sublimation/03_results/utils/helper_funcs.py:143: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/home/dlhogan/GitHub/Synoptic-Sublimation/03_results/utils/helper_funcs.py:143: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [32]:
fig = make_subplots(rows=4, cols=2,
                    specs=[[{}, {"rowspan": 2,
                                 "type":"polar"}],
                        [{}, None],
                        [{}, {"rowspan":2,
                              "type":"polar"}],
                        [{}, None]],
                    shared_xaxes=True,
                    vertical_spacing=0.1, 
                    subplot_titles=('Mean daily temperature', 
                                    'Wind direction frequency (all wind speeds) <br> ',
                                    'Mean daily vapor pressure deficit',
                                    'Cumulative precipitation',
                                    'Wind direction frequency (>5 m/s) <br> ',
                                    'Cumulative sublimation'))

# Mean Temperature Plot
fig.add_trace(go.Scatter(x=w22_mean_temperture.index, 
                         y=w22_mean_temperture['temp_mean'], 
                         name='2021-2022',
                         marker_color='red',
                         showlegend=False,
                         opacity=0.25),
                         row=1, col=1)
# add the total mean
fig.add_trace(go.Scatter(x=w22_mean_temperture.index,
                            y=w22_mean_temperture['temp_mean'].mean()*np.ones(len(w22_mean_temperture)),
                            name='2021-2022',
                            marker_color='red',
                            showlegend=True,
                            opacity=1.0,
                            line=dict(dash='solid',
                                      width=2)),
                            row=1, col=1)
fig.add_trace(go.Scatter(x=w23_mean_temperture.index,
                            y=w23_mean_temperture['temp_mean'],
                            name='2022-2023',
                            marker_color='black',
                            showlegend=False,
                            opacity=0.25),
                            row=1, col=1)
# add the total mean
fig.add_trace(go.Scatter(x=w23_mean_temperture.index,
                            y=w23_mean_temperture['temp_mean'].mean()*np.ones(len(w23_mean_temperture)),
                            name='2022-2023',
                            marker_color='black',
                            showlegend=True,
                            opacity=1.0,
                            line=dict(dash='solid',
                                      width=2)),
                            row=1, col=1)

# Mean Vapor Pressure Plot
fig.add_trace(go.Scatter(x=w22_mean_vapor_pressure_deficit.index, 
                         y=w22_mean_vapor_pressure_deficit['vapor_pressure_deficit'], 
                         name='2021-2022',
                         marker_color='red',
                         showlegend=False,
                         opacity=0.25),
                         row=2, col=1)
# add the total mean
fig.add_trace(go.Scatter(x=w22_mean_vapor_pressure_deficit.index,
                            y=w22_mean_vapor_pressure_deficit['vapor_pressure_deficit'].mean()*np.ones(len(w22_mean_vapor_pressure_deficit)),
                            name='2021-2022 Mean',
                            marker_color='red',
                            showlegend=False,
                            line=dict(dash='solid',
                                      width=2)),
                            row=2, col=1)   
fig.add_trace(go.Scatter(x=w23_mean_vapor_pressure_deficit.index,
                            y=w23_mean_vapor_pressure_deficit['vapor_pressure_deficit'],
                            name='2022-2023',
                            marker_color='black',
                            opacity=0.25,
                            showlegend=False,),
                            row=2, col=1)
# add the total mean
fig.add_trace(go.Scatter(x=w23_mean_vapor_pressure_deficit.index,
                            y=w23_mean_vapor_pressure_deficit['vapor_pressure_deficit'].mean()*np.ones(len(w23_mean_vapor_pressure_deficit)),
                            name='2022-2023 Mean',
                            marker_color='black',
                            showlegend=False,
                            line=dict(dash='solid',
                                      width=2)),
                            row=2, col=1)  

# Cumulative Precipitation
fig.add_trace(go.Scatter(x=w22_cumulative_precip.index, 
                         y=w22_cumulative_precip, 
                         name='2021-2022',
                         marker_color='red',
                         showlegend=False,),
                         row=3, col=1)
fig.add_trace(go.Scatter(x=w23_cumulative_precip.index,
                            y=w23_cumulative_precip,
                            name='2022-2023',
                            marker_color='black',
                            showlegend=False,),
                            row=3, col=1)
# Cumulative Sublimation
fig.add_trace(go.Scatter(x=w22_cumulative_sublimation.index,
                            y=w22_cumulative_sublimation,
                            name='2021-2022',
                            marker_color='red',
                            showlegend=False,),
                            row=4, col=1)
fig.add_trace(go.Scatter(x=w23_cumulative_sublimation.index,
                            y=w23_cumulative_sublimation,
                            name='2022-2023',
                            marker_color='black',
                            showlegend=False,),
                            row=4, col=1)

# Wind Rose 2022
fig.add_trace(
    go.Barpolar(
        r=w22_wind_rose_dir_df['frequency'].values,
        theta=w22_wind_rose_dir_df['direction'].values,
        marker_color='red',
        opacity=0.5,
        name='',
        showlegend=False,
    ),
    row=1, col=2
)

# Wind Rose 2023
fig.add_trace(
    go.Barpolar(
        r=w23_wind_rose_dir_df['frequency'].values,
        theta=w23_wind_rose_dir_df['direction'].values,
        marker_color='black', #marker_color is a function of radius
        opacity=0.5,
        showlegend=False,
    ),
    row=1, col=2
)

# Add high wind data

fig.add_trace(
    go.Barpolar(
        r=w22_wind_rose_spd_df['frequency'].values,
        theta=w22_wind_rose_spd_df['direction'].values,
        marker_color='red', #marker_color is a function of radius
        opacity=0.5,
        showlegend=False,
    ),
    row=3, col=2
)
# Wind Rose 2023

fig.add_trace(
    go.Barpolar(
        r=w23_wind_rose_spd_df['frequency'].values,
        theta=w23_wind_rose_spd_df['direction'].values,
        marker_color='black',
        opacity=0.5,
        name='',
        showlegend=False,
    ),
    row=3, col=2
)

axis1_range = [0,w22_wind_rose_dir_df[['frequency','direction']].groupby('direction', observed=True).sum().max().round(-1).item() + 10]
axis1_tickvals = np.arange(0, w22_wind_rose_dir_df[['frequency','direction']].groupby('direction', observed=True).sum().max().round(-1).item() + 10, 10).astype(int)
axis1_ticktext = [f'{val}%' for val in axis1_tickvals]

axis2_range = [0,w22_wind_rose_spd_df[['frequency','direction']].groupby('direction', observed=True).sum().max().round(-1).item()]
axis2_tickvals = np.arange(0, w22_wind_rose_spd_df[['frequency','direction']].groupby('direction', observed=True).sum().max().round(-1).item(), 5).astype(int)
axis2_ticktext = [f'{val}%' for val in axis2_tickvals]

fig.update_layout(
    title = '',
    yaxis1_title='Temperature<br>(C)',
    yaxis2_title='Vapor Pressure Deficit<br>(Pa)',
    yaxis3_title='Precipitation<br>(mm)',
    yaxis4_title='Sublimation<br>(mm)',
    xaxis4_title='Day of Year',
    polar1=dict(
        barmode='overlay',
        radialaxis_tickfont_size=18,
        radialaxis_range=axis1_range,
        radialaxis_tickvals=axis1_tickvals,
        radialaxis_ticktext=axis1_ticktext,
        angularaxis=dict(
            tickmode='array',
            rotation=90,
            direction='clockwise'
        )),
    polar2=dict(
        barmode='overlay',
        radialaxis_tickfont_size=18,
        radialaxis_range=axis2_range,
        radialaxis_tickvals=axis2_tickvals,
        radialaxis_ticktext=axis2_ticktext,
        angularaxis=dict(
            tickmode='array',
            rotation=90,
            direction='clockwise'
        )),
    legend1_title='Winter',
    # update the ylims
    yaxis1_range=[-20, 5],
    # yaxis2_range=[0, 40],
    width=1000,
    height=900,
)
# update the xaxis to show months
fig.update_xaxes(
    ticktext=['Dec','Jan','Feb','Mar', 'Apr'],
    tickvals=[61, 92, 120, 151, 181],
    # set xlim to end at 215
    range=[61,181],
)
# move legend to the top
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.07,
    xanchor="right",
    x=0.4,
    font_size=14,
))
# format the hover text
fig.update_traces(hovertemplate='Day of Water Year: %{x}<br>Wind Speed: %{y:.2f} m/s', row=1, col=1)
save_figure = True
if save_figure:
    # save figure as png
    fig.write_image(f'../04_products/figures/draft/w22_w23_met_comparison.png')    